In [1]:
import pandas as pd
import pickle as pk
import numpy as np

In [ ]:
noise_dataset=pd.read_csv('./raw_data/noisedataset.csv')
# print(noise_dataset.head(2))

   sensor_1_sensor_2_Bx  sensor_1_sensor_2_By  sensor_1_sensor_2_Bz  \
0            -69.447914              3.739586           -338.900883   
1             -6.147915             -2.410411             25.067169   

   sensor_1_sensor_3_Bx  sensor_1_sensor_3_By  sensor_1_sensor_3_Bz  \
0            -41.833329             62.462510              8.296863   
1             -4.333331             -1.287497             31.286904   

   sensor_1_sensor_4_Bx  sensor_1_sensor_4_By  sensor_1_sensor_4_Bz  \
0             23.862511            107.604173            400.510031   
1             -4.187500             -0.095828             29.039996   

   sensor_1_sensor_5_Bx  ...  sensor_5_sensor_8_Bz_abnormal  \
0            -36.020829  ...                            1.0   
1              1.929168  ...                            1.0   

   sensor_6_sensor_7_Bx_abnormal  sensor_6_sensor_7_By_abnormal  \
0                            1.0                            0.0   
1                            1.0  

In [2]:
print(noise_dataset.columns)

Index(['sensor_1_sensor_2_Bx', 'sensor_1_sensor_2_By', 'sensor_1_sensor_2_Bz',
       'sensor_1_sensor_3_Bx', 'sensor_1_sensor_3_By', 'sensor_1_sensor_3_Bz',
       'sensor_1_sensor_4_Bx', 'sensor_1_sensor_4_By', 'sensor_1_sensor_4_Bz',
       'sensor_1_sensor_5_Bx',
       ...
       'sensor_5_sensor_8_Bz_abnormal', 'sensor_6_sensor_7_Bx_abnormal',
       'sensor_6_sensor_7_By_abnormal', 'sensor_6_sensor_7_Bz_abnormal',
       'sensor_6_sensor_8_Bx_abnormal', 'sensor_6_sensor_8_By_abnormal',
       'sensor_6_sensor_8_Bz_abnormal', 'sensor_7_sensor_8_Bx_abnormal',
       'sensor_7_sensor_8_By_abnormal', 'sensor_7_sensor_8_Bz_abnormal'],
      dtype='object', length=173)


In [3]:
print(len(noise_dataset))

1100736


观测到的x是noisedataset里面前84列数据，对应的异常标签是noisedataset里面的后84列，真实值是normalrefdata中对应目标磁铁x,y,z的数据

按照7:1:2的来划分成train，valid，test。评估性能，分类性能和重建性能（MAE,MSE）

In [17]:
x=noise_dataset.values[:,:84]
x_pos=noise_dataset.values[:,84:89]
y=noise_dataset.values[:,89:]
print(x.shape)
print(x_pos.shape)
print(y.shape)

(1100736, 84)
(1100736, 5)
(1100736, 84)


In [18]:
print(x)

[[ -69.447914      3.73958633 -338.90088296 ...  -27.62083352
   -20.56770882 -236.74319995]
 [  -6.14791476   -2.41041138   25.06716895 ...   -4.8208338
    -3.9177092     3.32077633]
 [ -22.04791629  -40.21041062 -157.88481546 ...   -5.7208339
   -18.76770958  -84.52521289]
 ...
 [  -0.62842775   11.99818698   -3.65551075 ...   -1.68888266
     7.26446036   -4.2352758 ]
 [   6.72157264    7.34818163   -0.9935101  ...    2.06111808
     6.66445807   -2.05727821]
 [   7.32157302    4.64818469    0.45849148 ...    2.66111891
     6.66445807   -2.29927879]]


In [19]:
print(x_pos)

[[  0.  -4.   0.   0.   0.]
 [  2. -10.   0.   0.   0.]
 [  2.  -5.   0.   0.   0.]
 ...
 [  3. -10.  -3.   0.   0.]
 [ -1. -10.  -3.   0.   0.]
 [ -2. -10.  -3.   0.   0.]]


In [20]:
print(y)

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
normaldataset=pd.read_csv('./raw_data/normaldataset.csv')
print(len(normaldataset))

normal_x=normaldataset.values[:,:84]
normal_x_pos=normaldataset.values[:,84:89]
normal_y=normaldataset.values[:,89:]
print(normal_x.shape)
print(normal_x_pos.shape)
print(normal_y.shape)
# print(normal_x)
# print(normal_x_pos)
# print(normal_y)

226981
(226981, 84)
(226981, 5)
(226981, 84)


In [23]:
print(normal_x)

[[ 5.83125026e+00  3.65624748e+00  9.93918564e-02 ...  9.48213125e-01
   4.96607413e+00 -4.15721349e+00]
 [ 5.36307954e+00  4.56183726e+00 -5.83060517e-01 ...  2.19714316e+00
   4.66425954e+00 -4.98289696e+00]
 [ 5.02953908e+00  5.00778032e+00 -9.89473665e-01 ...  3.12469030e+00
   4.37253827e+00 -5.39194176e+00]
 ...
 [ 1.16941718e+02 -2.11213706e+02  5.27373636e+01 ...  6.05842090e+01
  -8.20078784e+01  2.47492624e+00]
 [ 1.10777898e+02 -2.02616573e+02  5.94962885e+01 ...  5.84814467e+01
  -7.97235642e+01  6.57018289e+00]
 [ 1.05018758e+02 -1.96106252e+02  6.42293910e+01 ...  5.65017838e+01
  -7.84660741e+01  1.06047875e+01]]


In [24]:
print(normal_x_pos)

[[ -3.  -10.   -3.    0.    0. ]
 [ -3.  -10.   -2.9   0.    0. ]
 [ -3.  -10.   -2.8   0.    0. ]
 ...
 [  3.   -4.    2.8   0.    0. ]
 [  3.   -4.    2.9   0.    0. ]
 [  3.   -4.    3.    0.    0. ]]


In [25]:
print(normal_y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
normalrefdata=pd.read_csv('./raw_data/normalrefdata.csv')
x_groundtruth=normalrefdata.values[:,:84]
x_groundtruth_pos=normalrefdata.values[:,84:89]
print(x_groundtruth.shape)
print(x_groundtruth_pos.shape)

(637, 84)
(637, 5)


In [28]:
print(x_groundtruth)

[[ -65.74166436    5.17083412 -374.87821983 ...  -25.30833375
   -19.27083304 -249.7103665 ]
 [  -2.44166512   -0.97916359  -10.91016792 ...   -2.50833404
    -2.62083342   -9.64639021]
 [ -18.34166665  -38.77916283 -193.86215232 ...   -3.40833414
   -17.4708338   -97.49237944]
 ...
 [  -1.3687505    12.80625282   -4.49860772 ...   -2.05178688
     6.76607336   -4.88321142]
 [   5.98124988    8.15624748   -1.83660706 ...    1.69821387
     6.16607107   -2.70521383]
 [   6.58125026    5.45625053   -0.38460549 ...    2.2982147
     6.16607107   -2.94721441]]


In [29]:
print(x_groundtruth_pos)

[[  0.  -4.   0.   0.   0.]
 [  2. -10.   0.   0.   0.]
 [  2.  -5.   0.   0.   0.]
 ...
 [  3. -10.  -3.   0.   0.]
 [ -1. -10.  -3.   0.   0.]
 [ -2. -10.  -3.   0.   0.]]


数据集样本(x,label,x_tilde),x为N,C的矩阵，N=28，C=3，表示观测到的异常矩阵；label为N，C的矩阵，表示数值是否异常,1为异常，0为正常；x_tilde为N，C的矩阵，表示x的真实值矩阵

In [49]:
index_list=normalrefdata.index.tolist()
print(index_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [50]:
import numpy as np
data_dict={}
for i in range(637):
    data_dict[i]={}
    data_dict[i]['x_data']=[]
    data_dict[i]['label_data']=[]
    data_dict[i]['x_tilde']=[]
    
def search_x_tilde(x_pos_data,x_groundtruth):
    magnet_x=x_pos_data[0]
    magnet_y=x_pos_data[1]
    magnet_z=x_pos_data[2]
    match_x_tilde=x_groundtruth[(x_groundtruth['magnet_x'] == magnet_x) &
                                (x_groundtruth['magnet_y'] == magnet_y) &
                                (x_groundtruth['magnet_z'] == magnet_z)]
    index=match_x_tilde.index
    if match_x_tilde.shape[0]==1:
        match_x_tilde=match_x_tilde.values[0,:84]
    else:
        print('error')
        return None
    return match_x_tilde,index

for row_data in noise_dataset.values:
    x_data_item=row_data[:84]
    x_pos_data_item=row_data[84:87]
    y_data_item=row_data[89:]
    x_tilde_item,index=search_x_tilde(x_pos_data_item,normalrefdata)
    data_dict[index.values[0]]['x_data'].append(x_data_item)
    data_dict[index.values[0]]['label_data'].append(y_data_item)
    data_dict[index.values[0]]['x_tilde'].append(x_tilde_item)

In [61]:
# all_data=[x_data,label_data,x_tilde]
# import pickle as pk
# with open('processed_dataset/all_data.pkl','wb') as fb:
#     pk.dump(all_data,fb)
x_data,label_data,x_tilde=[],[],[]
for key,value in data_dict.items():
    print(value['x_data'].shape)
    break

(1728, 84)


In [56]:
N=1728

indices=np.random.permutation(N)

n_train=int(N*0.7)
n_valid=int(N*0.1)
n_test=int(N*0.2)

train_x_data,train_label_data,train_x_tilde=[],[],[]
valid_x_data,valid_label_data,valid_x_tilde=[],[],[]
test_x_data,test_label_data,test_x_tilde=[],[],[]

for key,value in data_dict.items():
    train_indices=indices[:n_train]
    valid_indices=indices[n_train:n_train+n_valid]
    test_indices=indices[n_train+n_valid:]
    value['x_data']=np.array(value['x_data'])
    value['label_data']=np.array(value['label_data'])
    value['x_tilde']=np.array(value['x_tilde'])
    train_x_data.extend(value['x_data'][train_indices])
    train_label_data.extend(value['label_data'][train_indices])
    train_x_tilde.extend(value['x_tilde'][train_indices])
    valid_x_data.extend(value['x_data'][valid_indices])
    valid_label_data.extend(value['label_data'][valid_indices])
    valid_x_tilde.extend(value['x_tilde'][valid_indices])
    test_x_data.extend(value['x_data'][test_indices])
    test_label_data.extend(value['label_data'][test_indices])
    test_x_tilde.extend(value['x_tilde'][test_indices])
train_x_data=np.array(train_x_data)
train_label_data=np.array(train_label_data)
train_x_tilde=np.array(train_x_tilde)
valid_x_data=np.array(valid_x_data)
valid_label_data=np.array(valid_label_data)
valid_x_tilde=np.array(valid_x_tilde)
test_x_data=np.array(test_x_data)
test_label_data=np.array(test_label_data)
test_x_tilde=np.array(test_x_tilde)
print(train_x_data.shape,train_label_data.shape,train_x_tilde.shape)
print(valid_x_data.shape,valid_label_data.shape,valid_x_tilde.shape)
print(test_x_data.shape,test_label_data.shape,test_x_tilde.shape)



(770133, 84) (770133, 84) (770133, 84)
(109564, 84) (109564, 84) (109564, 84)
(221039, 84) (221039, 84) (221039, 84)


In [57]:
import pickle as pk

train_data=[train_x_data,train_label_data,train_x_tilde]
valid_data=[valid_x_data,valid_label_data,valid_x_tilde]
test_data=[test_x_data,test_label_data,test_x_tilde]

with open('processed_dataset/train.pkl','wb') as fb:
    pk.dump(train_data,fb)
with open('processed_dataset/valid.pkl','wb') as fb:
    pk.dump(valid_data,fb)
with open('processed_dataset/test.pkl','wb') as fb:
    pk.dump(test_data,fb)
    


number of train samples:  1209
number of valid samples:  172
number of test samples:  347


In [58]:
print("number of train samples: ",len(train_x_data))
print("number of valid samples: ",len(valid_x_data))
print("number of test samples: ",len(test_x_data))

number of train samples:  770133
number of valid samples:  109564
number of test samples:  221039


In [60]:
train_data=pk.load(open('processed_dataset/train.pkl','rb'))
valid_data=pk.load(open('processed_dataset/valid.pkl','rb'))
test_data=pk.load(open('processed_dataset/test.pkl','rb'))
print(train_data[0].shape,train_data[1].shape,train_data[2].shape)
print(valid_data[0].shape,valid_data[1].shape,valid_data[2].shape)
print(test_data[0].shape,test_data[1].shape,test_data[2].shape)

(770133, 84) (770133, 84) (770133, 84)
(109564, 84) (109564, 84) (109564, 84)
(221039, 84) (221039, 84) (221039, 84)


In [12]:
import pandas as pd
import pickle as pk
import numpy as np

alltest=pd.read_csv('./raw_data/alltest.csv')

# alltest_x=alltest.values[:,:84]
# alltest_x_pos=alltest.values[:,84:87]
# alltest_label=alltest.values[:,89:]
# print(alltest_x.shape,alltest_label.shape,alltest_x_tilde.shape)

normalrefdata=pd.read_csv('./raw_data/normalrefdata.csv')

def search_x_tilde(x_pos_data,x_groundtruth):
    magnet_x=x_pos_data[0]
    magnet_y=x_pos_data[1]
    magnet_z=x_pos_data[2]
    match_x_tilde=x_groundtruth[(x_groundtruth['magnet_x'] == magnet_x) &
                                (x_groundtruth['magnet_y'] == magnet_y) &
                                (x_groundtruth['magnet_z'] == magnet_z)]
    # index=match_x_tilde.index
    if match_x_tilde.shape[0]==1:
        match_x_tilde=match_x_tilde.values[0,:84]
    else:
        print('error')
        return None
    return match_x_tilde


all_test_x_data,all_test_label_data,all_test_x_tilde=[],[],[]
for row_data in alltest.values:
    x_data_item=row_data[:84]
    x_pos_data_item=row_data[84:87]
    y_data_item=row_data[89:]
    x_tilde_item=search_x_tilde(x_pos_data_item,normalrefdata)
    all_test_x_data.append(x_data_item)
    all_test_label_data.append(y_data_item)
    all_test_x_tilde.append(x_tilde_item)
all_test_x_data=np.array(all_test_x_data)
all_test_label_data=np.array(all_test_label_data)
all_test_x_tilde=np.array(all_test_x_tilde)
print(all_test_x_data.shape,all_test_label_data.shape,all_test_x_tilde.shape)

(366275, 84) (366275, 84) (366275, 84)


In [13]:
all_test_data=[all_test_x_data,all_test_label_data,all_test_x_tilde]
with open('processed_dataset/all_test.pkl','wb') as fb:
    pk.dump(all_test_data,fb)


In [14]:
hardiron_test=pd.read_csv('./raw_data/hardiron_test.csv')
# print(len(hardiron_test))

hardiron_test_x_data,hardiron_test_label_data,hardiron_test_x_tilde=[],[],[]
for row_data in hardiron_test.values:
    x_data_item=row_data[:84]
    x_pos_data_item=row_data[84:87]
    y_data_item=row_data[89:]
    x_tilde_item=search_x_tilde(x_pos_data_item,normalrefdata)
    hardiron_test_x_data.append(x_data_item)
    hardiron_test_label_data.append(y_data_item)
    hardiron_test_x_tilde.append(x_tilde_item)
hardiron_test_x_data=np.array(hardiron_test_x_data)
hardiron_test_label_data=np.array(hardiron_test_label_data)
hardiron_test_x_tilde=np.array(hardiron_test_x_tilde)
print(hardiron_test_x_data.shape,hardiron_test_label_data.shape,hardiron_test_x_tilde.shape)

(183456, 84) (183456, 84) (183456, 84)


In [15]:
hardiron_test_data=[hardiron_test_x_data,hardiron_test_label_data,hardiron_test_x_tilde]
with open('processed_dataset/hardiron_test.pkl','wb') as fb:
    pk.dump(hardiron_test_data,fb)

In [16]:
softiron_test=pd.read_csv('./raw_data/softiron_test.csv')
# print(len(softiron_test))

softiron_test_x_data,softiron_test_label_data,softiron_test_x_tilde=[],[],[]
for row_data in softiron_test.values:
    x_data_item=row_data[:84]
    x_pos_data_item=row_data[84:87]
    y_data_item=row_data[89:]
    x_tilde_item=search_x_tilde(x_pos_data_item,normalrefdata)
    softiron_test_x_data.append(x_data_item)
    softiron_test_label_data.append(y_data_item)
    softiron_test_x_tilde.append(x_tilde_item)
softiron_test_x_data=np.array(softiron_test_x_data)
softiron_test_label_data=np.array(softiron_test_label_data)
softiron_test_x_tilde=np.array(softiron_test_x_tilde)
print(softiron_test_x_data.shape,softiron_test_label_data.shape,softiron_test_x_tilde.shape)


(182819, 84) (182819, 84) (182819, 84)


In [ ]:
softiron_test_data=[softiron_test_x_data,softiron_test_label_data,softiron_test_x_tilde]
with open('processed_dataset/softiron_test.pkl','wb') as fb:
    pk.dump(softiron_test_data,fb)

: 

train_data: 训练数据
test_data: 测试数据
harditron_test_data, softiron_test_data: 测试数据
样本：(X,label,X_tilde), X表示观测的异常样本，label标识观测值是否异常，X_tilde表示X对应的正常样本(Groundtruth)

In [2]:
train_data=pk.load(open('processed_dataset/train_data.pkl','rb'))
test_data=pk.load(open('processed_dataset/test_data.pkl','rb'))
print(train_data[0].shape,train_data[1].shape,train_data[2].shape)
print(test_data[0].shape,test_data[1].shape,test_data[2].shape)

(1100736, 84) (1100736, 84) (1100736, 84)
(366275, 84) (366275, 84) (366275, 84)


In [3]:
hardiron_test_data=pk.load(open('processed_dataset/hardiron_test.pkl','rb'))
print(hardiron_test_data[0].shape,hardiron_test_data[1].shape,hardiron_test_data[2].shape)
softiron_test_data=pk.load(open('processed_dataset/softiron_test.pkl','rb'))
print(softiron_test_data[0].shape,softiron_test_data[1].shape,softiron_test_data[2].shape)

(183456, 84) (183456, 84) (183456, 84)
(182819, 84) (182819, 84) (182819, 84)


In [2]:
from dataloader.data_provider import *

train_dataloader=get_dataloader('processed_dataset/',mode="train",batch_size=32)
valid_dataloader=get_dataloader('processed_dataset/',mode="val",batch_size=32)
test_dataloader=get_dataloader('processed_dataset/',mode="test",batch_size=32)

print("len(train_dataloader): ",len(train_dataloader))
print("len(valid_dataloader): ",len(valid_dataloader))
print("len(test_dataloader): ",len(test_dataloader))

len(train_dataloader):  27519
len(valid_dataloader):  6880
len(test_dataloader):  11447


In [4]:
for _,batch_data in enumerate(train_dataloader):
    x,label,x_tilde=batch_data
    break
print(x.shape,label.shape,x_tilde.shape)

torch.Size([32, 84]) torch.Size([32, 84]) torch.Size([32, 84])
